<a href="https://colab.research.google.com/github/Stephan-Dupoux/DSCI-691-Project/blob/main/Presentation_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mining Adverse Drug Events from Tweets
### [Project Repository](https://github.com/Stephan-Dupoux/DSCI-691-Project)

*Group Members:* 
1. Layla Bouzoubaa - lb3338@drexel.edu 
2. Stephan Dupoux - sgd45@drexel.edu
3. Hannah Wurzel - hjw35@drexel.edu
4. Zifeng Wang - zw438@drexel.edu 


### **Introduction**

Adverse drug reactions (ADR) are described as “harmful reactions that are caused by the intake of medication”. In fact, 3%-7% of hospitalizations occur because of ADR. Additionally, 10%-20% of hospital patients will endure ADR while hospitalized. It is thought that nearly 10%-20% of these cases of ADR are severe and can lead to damaging side effects and even death in some cases. Since all drugs have the potential for adverse drug reactions, risk-benefit analysis (analyzing the likelihood of benefit vs risk of ADRs) is necessary whenever a drug is prescribed. On the other hand, incidence and severity of ADR may vary by patient characteristics and drug factors, which makes it a quite time-consuming and challenging to task to detect ADR from traditional medicine consumer reports or even modern electronic health records.  

With the prosperity of social networks, people are more inclined to share their treatment experiences on social media, posting their use of prescription drugs and related side effects. This behavior makes user posts on social media an important source for ADR detection. User posts discussing ADRs must be identified before further detection can occur because almost 90% of drug-related user posts are not associated with ADRs. Detecting the presence of an ADR in each user post is a key of success of further data mining. 

On the other hand, mining tweets in general has been an issue for NLP specialists for quite some time.  Due to the relaxed nature of tweets they tend to contain a significant amount of noise.  This noise can be from users writing in slang, the use of emojis, misspelled words, etc.  This noise is difficult to deal with because it oftentimes is vital in understanding the message the user is attempting to get across.

The overarching goal of this project is to develop a basic framework of mining the value of social media postings as a source of “signals” of potential ADR, paying particular attention at the value such information might have to detect adverse events earlier than currently possible, and to detect effects not easily captured by traditional means. 

#### Baseline

In addition to the literature, the first homework assignment provided a strong baseline from which to develop our classifier.  Similar to the problem we are looking to solve, homework one required the creation of a binary classifier to determine if tweets were in a chunk of text or not. Lines of code from the first homework assignment and `01-utilities.py` were re-purposed to fit our needs.


#### Data

Most of the data was provided to us by the organizers of the SMM4H 2022.  This data includes nearly 17,500 labeled tweets which will be used for training.  From these tweets, 1,235 are classified as ADR.  SMM4H also provides participants with around 10,000 tweets which will be used for testing. SMM4H organizers will be providing a validation and test set in the near future.

##### Code

To be mindful of the training time and computational resources required to perform the experiments, only code snippets and outcomes are reported below. Links to scripts within the text are provided for the full code.

#### Task

The Social Media Mining for Health Applications (#SMM4H) is a yearly NLP challenge held by the Health Language Processing Lab at University of Pennsylvania.  Each year they devise ten different tasks and accompanying subtasks related to NLP in the health sector. Our group chose to complete task #1: Classification, detection and normalization of Adverse Events (AE) mentions in tweets (in English). Our group selected the first two subtasks:
1. Classify tweets reporting ADEs (Adverse Drug Events)
2. Detect ADE spans in tweets

Our group was interested in this task because of the concerning statistics surrounding ADEs. In fact, 3%-7% of hospitalizations occur because of an ADE.  Additionally, 10%-20% of hospital patients will endure an ADE while hospitalized.  It is thought that nearly 10%-20% of ADE cases are severe and can lead to damaging side effects and even death<sup>1</sup>.

Observing the tweets, one could determine an ADE was present without much uncertainty, as in this tweet:  
> "okay i took the seroquel and i am resisting the urge to vomit it up i feel disgusting but oh well good times good vibes don't be sad"  

However, in some of the tweets labeled as having an ADE, it is not so clear:  
> "@USER_____ i'm not lazy!! my seroquel knocked me out!!"  

One could postulate that the inclusion of the phrases like "knocked me out" or "not lazy" would classify this as having an ADE but we cannot know for sure. This slight ambiguity is what makes this task interesting and challenging. It is worth observing the positionality of the tweets to facilitate the development of the classification and extraction pipeline. 


### Preprocessing 

We wanted to keep the tweets closest to their original form as possible.  However, there was some noise in these tweets that we decided to remove.  This includes the user mentions ('@USER'), any proceeding "_" from tweets, emojis, and stopwords. ADE labels were converted from character to binary; `ADE`:`1`, `NoADE`:`0`.  


```
def clean_tweet(tweet, remove_stopwords = True):
    # lowercase and remove whitespace
    new = tweet.lower().strip()
    # remove any instance of '@USER' followed by '_'
    new = re.sub(r'@\w+_', '', new)
    # remove extranous characters
    new = re.sub(r'[^\w\s]', ' ', new).strip()
    # remove emojis
    new = new.encode('ascii', errors='ignore').decode('utf8').strip()
    # remove stopwords
    if remove_stopwords:
        new = ' '.join([word for word in new.split() if word not in stop_words])
    
    return new
  
def tokenize(text, space = False, clean = True):
    if clean: text = clean_tweet(text)
    tokens = []
    for token in re.split("([0-9a-zA-Z'-]+)", text):
        if not space:
            token = re.sub("[ ]+", "", token)
        if not token:
            continue
        if re.search("[0-9a-zA-Z'-]", token):                    
            tokens.append(token)
        else: 
            tokens.extend(token)
    return tokens

```

### Models and Results

#### **Subtask 1**
A training set, which made up of 80% of the data, created. The remaining 20% was reserved as a test set. When examining the training set, there was an observable imbalance between tweets that contain an ADE and those that do not (Figure 1).

![Figure 1](./img/train-dist.png)

#### Feature based models

[SEE CODE HERE](./python/baseline_models_a.ipynb)

Logistic regression and support vector machines (SVM) were chosen as the baseline models from which to compare additional hyper tuned and engineered models as well as the proceeding neural models.

Both logistic and SVM models were based on TF-IDF features. Because TF-IDF represents the degree of salience, as it balances the frequency of a word with the number of documents it appears in, it was selected over a bag-of-words (BOW) representation.

1. Logistic Regression

  1. vectorize training, test
  2. calculate tf-idf
  3. fit logistic regression with balanced weights
  4. apply model to predict classes

  This `sklearn` implementation of a logistic classifier resulted in the following performance metrics:

  ```
AUC: 0.818335056178608
Precision: 0.40
Recall: 0.72
F1 Score: 0.51

  ```

2. SVM

  1. SVM model with linear kernel and balanced weights on tf-idf features

  The `sklearn` implementation of the SVM classifier resulted in the following performance metrics:

  ```
  AUC: 0.7773205167496922
  Precision: 0.47
  Recall: 0.61
  F1 Score: 0.53
  ```

As these baselines out-performs that of Rawal et al.<sup>2</sup> (F1: 0.4317, Precision:0.3223, Recall: 0.6534), we are optimistic our subsequent models will follow this trend.


#### Word Embedding

For this task, utilizing a word embedding allows us to better assess which words are likely to be found together. Experimentation using pre-trained embeddings, specifically Google's [word2vec](https://en.wikipedia.org/wiki/Word2vec) (2013) and Stanford's [GloVe](https://en.wikipedia.org/wiki/GloVe_(machine_learning)) (2014), were performed but resulted in lower than baseline F1, Recall, and Precision scores. We decided to train our own word2vec model with our training corpus using the `gensim` library. For this step, we used unigrams, however we intend to experiment with bi-grams and tri-grams in a future experiments. The results of this step showed promising results.

Similarity scores with *'drug'*:
```
[('b', 0.9992734789848328),
 ('made', 0.998689591884613),
 ('disapear', 0.9974077939987183),
 ('corcet', 0.9970759153366089),
 ('enbrel', 0.995078444480896),
 ('mickelson', 0.9944359064102173),
 ('investigating', 0.9930176138877869),
 ('information', 0.992373526096344),
 ('issued', 0.9907057881355286),
 ('biosimilar', 0.9902804493904114)]
```

Similarity scores with *'dying'*:
```
[('medicine', 0.9977312088012695),
 ('starts', 0.997506320476532),
 ('latuda', 0.9974697232246399),
 ('quit', 0.997446596622467),
 ('disorder', 0.9973891377449036),
 ('ask', 0.9971755743026733),
 ('pain', 0.9971699714660645),
 ('allergic', 0.9969409704208374),
 ('asking', 0.9968919157981873),
 ('combination', 0.9967824220657349)]


```

Similarity scores with *'seroquel'*:
```
[('adderall', 0.9993433952331543),
 ('every', 0.9993388652801514),
 ('gonna', 0.9991791844367981),
 ('put', 0.9989113211631775),
 ('hate', 0.9987514019012451),
 ('going', 0.9986067414283752),
 ('xanax', 0.9985816478729248),
 ('took', 0.9984757304191589),
 ('god', 0.9984127283096313),
 ('im', 0.998390793800354)]
```

These experiments with word embeddings led us to believe that there is significant potential that could be harnessed with our neural architectures. The multiple layers in a deep learning architecture makes it more suitable for handing the task of "disentagling" dense input data such as neural word embeddings.


#### Neural Architecture

1. Baseline: Multilayer Perceptron

  Initial experimentation for a baseline neural framework used a multilayered perceptron (MLP) model. MLPs are markedly different in terms of how it computes predictions and how it incorporates data into the models compared to the Bi-LSTM. This baseline Multilayer perceptron model is a simple feed forward network model. The Multi Layered Perceptron model was built with the scikit learn model. As seen with the picture in the code used an adams optimizer.

```
Precision of .53
Recall of .4
F1 of .46
ROC_AUC of .6867
```

RE: word embeddings
> Hidden layers in neural networks act like kernels in disentangling linearly inseparable input data layer by layer, step by step<sup>3</sup>


2. Bi-LSTM

The BI-LSTM was another neural architecture that we wanted to use for our experiments. The Bi-LSTM architecture is marketed with one recursive layer and one time distributed layer. This model also uses the same preprocessing techniques with the other Neural Network Model. 

What makes this process interesting is that the process with the neural network model id not the specific usage for this type of algorithm. This was done intentionally as to see the predictive power of the models and setting a first step before doing more feature engineering with this model.

```
Bi-LSTM with TF-IDF preprocessing
Precision of .410
Recall of .405
F1 of .407
```

#### FF MLP 

[SEE CODE HERE](./python/ff-mlp.py)

```
Multilayer Perceptron: with sigmoid activation
AUC: 0.6867425757489182
Precision: 0.53
Recall: 0.40
F1 Score: 0.46

```

### Discussion



#### Challenges

There were a few challenges our group dealt with while going through the process of trying to solve these tasks.  The first issue we ran into was that our data is extremely unbalanced.  Specifically, 7.2% data points are classified as `ADE` while the remaining 92.8% are classified as `NoADE`.  Unbalanced data like this leads to skeweness that we are adjusting for in our models with continued tuning and feature engineering.

Working with tweets, there are certain elements that could be considered noise. This includes usernames, slang, emojis, etc.  To preserve the integrity of the context within the tweets, we removed only usernames and emojis and extraneous characters, like slashes and ellipses, that we believe do not add much value to model performance.

The last challenge was in regard to our data's vocabulary. The verbiage used in the tweets are euphemistic, include misspellings, and primarily pertain to an experience or reference to a drug. This makes the term-frequency matrix sparse. Attempting to utilize a pre-trained word embedding, like GloVe, resulted in poor performance in the SVM and LR models. We believe this may be due to the introduction of several 0-vectors due to out-of-vocabulary terms.

We intend on addressing these challenges as we continue our work on these tasks.

#### Next Steps

Work on these tasks is an ongoing process. Similar studies that utilized roBERTa in their architecture have reported favorable results. Future work will include experimenting with transformer-based language models, domain-specific lexicons, and leveraging transfer learning.

There are many lexicons available for this task.  We will be experimenting with the Unified Medical Language System (UMLs), MedDRA, or DrugBank. [RedMed](https://doi.org/10.1016/j.jbi.2019.103307)<sup>4</sup> is a word embedding model that is used to identify and retrieve health data from reddit posts.  We believe that we could implement RedMed on our models to perform well with twitter data in detecting ADEs.

Once we establish a satisfactory engine for classifying tweets, we will leverage the knowledge learned to extract the spans of text containing the ADE. Doing so can significantly reduce computing resources and effort, particularly when working with large language models (i.e. BERT), when done correctly. Putting together these two tasks into a single pipeline will reduce the amount of data required to achieve the same performance as if the tasks were trained in parallel. 

### References

1. Marsh, D. E. S. (2022, June 2). Adverse drug reactions - clinical pharmacology. Merck Manuals Professional Edition. Retrieved June 6, 2022, from https://www.merckmanuals.com/professional/clinical-pharmacology/adverse-drug-reactions/adverse-drug-reactions 
2. [Identification of Adverse Drug Reaction Mentions in Tweets – SMM4H Shared Task 2019](https://aclanthology.org/W19-3225) (Rawal et al., 2019)
3. Chollet, F. (2017). Deep learning with python. Manning Publications.
4. Lavertu, A. & Altman, R. B. "RedMed: Extending drug lexicons for social media applications"
Journal of Biomedical Informatics, (2019)